PromptTemplate是帮助我们定义一个包含变量的字符串模板，我们可以通过向该类的对象输入不同的变量值来生成模板渲染的结果，意思就是在运行时可以根据用户的输入有确定的输出。

In [1]:
//不含变量的template
import { PromptTemplate } from "@langchain/core/prompts";

const greetingPrompt = new PromptTemplate({
    inputVariables:[],
    template:"Hello, World!"
})

const formattedGreetingPrompt = await greetingPrompt.format();

console.log(formattedGreetingPrompt);


Hello, World!


调用promptTemplate的方式就是format，因为我们没有任何变量，也就没有任何参数


In [2]:
//含变量的template

const personalizedGreetingPrompt = new PromptTemplate({
    inputVariables: ["name"],
    //如果需要{}或者{{}}，需要转义
    template: "hello，{name} {{name}}",
  });
  const formattedPersonalizedGreeting = await personalizedGreetingPrompt.format({
    name: "TH",
  });
  
  console.log(formattedPersonalizedGreeting);

hello，TH {name}


In [3]:
//langchain也提供了简便的创建方式
import { PromptTemplate } from "@langchain/core/prompts";
const autoInferTemplate = PromptTemplate.fromTemplate("good {name}")
console.log(autoInferTemplate.inputVariables);

const format = await autoInferTemplate.format({
    name:"TG"
})

console.log(format);



good TG


在工程中，我们可能先获得某个参数，之后才能获得另一个参数

In [4]:
import { PromptTemplate } from "@langchain/core/prompts";

const initialPrompt = new PromptTemplate({
    template:"这是一个{type} 他是{item}",
    inputVariables:["type","item"]
})

const partialedPrompt = await initialPrompt.partial({
    type:"工具"
})

const formattedPrompt = await partialedPrompt.format({
    item:'锤子'
})

console.log(formattedPrompt);





这是一个工具 他是锤子


### 使用动态填充参数
当我们需要，一个prompt template被format时，实时地动态生成参数时，我们可以使用函数来对template部分参数进行指定。

In [5]:
const getCurrentDateStr = () => {
    return new Date().toLocaleDateString();
  };
  
  const promptWithDate = new PromptTemplate({
    template: "今天是{date}，{activity}。",
    inputVariables: ["date", "activity"],
  });
  
  const partialedPromptWithDate = await promptWithDate.partial({
    date: getCurrentDateStr,
  });
  
  const formattedPromptWithDate = await partialedPromptWithDate.format({
    activity: "我们去爬山",
  });
  
  console.log(formattedPromptWithDate);
  // 输出: 今天是2023/7/13，我们去爬山。
  

今天是6/8/2024，我们去爬山。


注意，函数 getCurrentDateStr 是在 format 被调用的时候实时运行的，也就是可以在被渲染成字符串时获取到最新的外部信息。 目前这里不支持传入参数，如果需要参数，可以用 js 的闭包进行参数的传递。

In [6]:
const getCurrentDateStr = () => {
    return new Date().toLocaleDateString();
  };
  
  function generateGreeting(timeOfDay) {
    return () => {
      const date = getCurrentDateStr()
      switch (timeOfDay) {
        case 'morning':
          return date + ' 早上好';
        case 'afternoon':
          return date + ' 下午好';
        case 'evening':
          return date + ' 晚上好';
        default:
          return date + ' 你好';
      }
    };
  }
  
  const prompt = new PromptTemplate({
    template: "{greeting}!",
    inputVariables: ["greeting"],
  });
  
  const currentTimeOfDay = 'afternoon';
  const partialPrompt = await prompt.partial({
    greeting: generateGreeting(currentTimeOfDay),
  });
  
  const formattedPrompt = await partialPrompt.format();
  
  console.log(formattedPrompt);
  // 输出: 3/21/2024 下午好!
  

6/8/2024 下午好!


在跟各种聊天模型交互的时候，在构建聊天信息时，不仅仅包含了像上文中的文本内容，也需要与每条消息关联的角色模型，例如这条信息时由人类、AI、还是给chatbot指定的system信息，这种结构化的消息输入有助于模型更好理解对话的上下文和流程，从而生成更准确、更自然的回应。

为了方便的构建和处理这种结构化的聊天消息，langchain提供了几种与聊天相关的提示模板类，如
 * ChatPromptTemplate
 * SystemMessagePromptTemplate
 * AIMessagePromptTemplate
 * HumanMessagePromptTemplate

在openAI的定义中，每一条消息都需要跟一个role关联，标识消息的发送者。角色的概念对于LLM的理解和构建整个对话流程非常重要，相同的内容由不同的role发送出来的意义是不同的

* system角色的消息通常用于设置对话的上下文或指定模型采取特定的行为模式，这些消息不会直接显示在对话中，但他们对模型的行为有指导作用，可以理解成模型的元信息，权重非常高，在这里有效的构建prompt能取得非常好的效果

* user角色代表真实用户在对话中的发言，这些消息通常是问题，指令或者评论，反映了用户的意图和需求。

* assistant角色的消息代表AI模型的回复，这些消息时模型根据system的指示和user的输入生成的



我们先构建一个system message来给llm指定核心的准则

In [7]:
import { SystemMessagePromptTemplate } from "@langchain/core/prompts";
import { HumanMessagePromptTemplate } from "@langchain/core/prompts";
import { ChatPromptTemplate } from "@langchain/core/prompts";

import { load } from "dotenv";
import { ChatOpenAI } from "@langchain/openai";
import { StringOutputParser } from "@langchain/core/output_parsers";

const translateInstructionTemplate = SystemMessagePromptTemplate.fromTemplate(`你是一个专业的翻译
员，你的任务时将文本从{source_lang}翻译成{target_lang}。`);

//然后构建一个用户输入的信息
const userQuestionTemplate = HumanMessagePromptTemplate.fromTemplate("请翻译这句话：{text}")


//然后将这两个信息组合起来，形成一个对话信息
const chatPrompt = ChatPromptTemplate.fromMessages([
  translateInstructionTemplate,
  userQuestionTemplate,
]);

const res = await chatPrompt.format({
    source_lang: "中文",
    target_lang: "英语",
    text: "你好，世界",
  });

  console.log(res);
  

//然后我们就可以快速组装起一个简单的 chain 来测试一下
const env = await load();
const process = {
    env
}

const chatModel = new ChatOpenAI({
  model:'gpt-3.5-turbo',
  configuration:{
      baseURL: "https://ai.nvtnet.net/v1",
  }
}
);
const outputPraser = new StringOutputParser();

const chain = chatPrompt.pipe(chatModel).pipe(outputPraser);

await chain.invoke({
  source_lang: "中文",
  target_lang: "法语",
  text: "你好，世界",
})
// "Bonjour, le monde"



System: 你是一个专业的翻译
员，你的任务时将文本从中文翻译成英语。
Human: 请翻译这句话：你好，世界


"Bonjour, monde."

### 组合多个prompt
在实际工程中，我们可能会根据多个变量，根据多个外界环境去构造一个很复杂的prompt，这里就是PipelinePromptTemplate 的应用场景，我可以用多个独立的template构建成一个完成且复杂的prompt，这样可以提高独立prompt的复用性，进一步增强模块化带来的优势

在PipelinePromptTemplate有2个核心的概念

* pipelinePrompts，一组object 每个object标识prompt运行后赋值给name变量
* finalPrompt，表示最终输出的prompt

In [8]:
import {
    PromptTemplate,
    PipelinePromptTemplate,
  } from "@langchain/core/prompts";
  
  const getCurrentDateStr = () => {
    return new Date().toLocaleDateString();
  };
  
  const fullPrompt = PromptTemplate.fromTemplate(`
  你是一个智能管家，今天是 {date}，你的主人的信息是{info}, 
  根据上下文，完成主人的需求
  {task}`);
  
  const datePrompt = PromptTemplate.fromTemplate("{date}，现在是 {period}")
  const periodPrompt = await datePrompt.partial({
      date: getCurrentDateStr
  })
  
  const infoPrompt =  PromptTemplate.fromTemplate("姓名是 {name}, 性别是 {gender}");
  
  const taskPrompt = PromptTemplate.fromTemplate(`
  我想吃 {period} 的 {food}。 
  再重复一遍我的信息 {info}`);
  
  const composedPrompt = new PipelinePromptTemplate({
    pipelinePrompts: [
      {
        name: "date",
        prompt: periodPrompt,
      },
      {
        name: "info",
        prompt: infoPrompt,
      },
      {
        name: "task",
        prompt: taskPrompt,
      },
    ],
    finalPrompt: fullPrompt,
  });
  
  const formattedPrompt = await composedPrompt.format({
      period: "早上",
      name: "张三",
      gender: "male",
      food: "lemon"
  });
  
  console.log(formattedPrompt)
  


  你是一个智能管家，今天是 6/8/2024，现在是 早上，你的主人的信息是姓名是 张三, 性别是 male, 
  根据上下文，完成主人的需求
  
  我想吃 早上 的 lemon。 
  再重复一遍我的信息 姓名是 张三, 性别是 male


一个变量可以多次复用，例如外界输入的 period 在 periodPrompt 和 taskPrompt 都被使用了
pipelinePrompts 中的变量可以被引用，例如我们在 taskPrompt 使用了 infoPrompt 的运行结果
支持动态自定义和 partial。例子中我们也涉及到了这两种特殊的 template
langchain 会自动分析 pipeline 之间的依赖关系，尽可能的进行并行化来提高运行速度
有了 pipelinePrompts 我们可以极大程度的复用和管理我们的 prompt template，从而让 llm app 的开发更加工程化。